<a href="https://colab.research.google.com/github/papercore-dev/perfectly-jogyo/blob/main/vc-run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **⚠ GPU나 TPU를 이용한 비트코인 및 가상화폐 채굴은 금지되어 있습니다. ⚠**

# 🐸TTS × YourTTS
## VITS 기반 TTS 엔진 제작기

제가 장담하건대, 저 포함 여러분들의 99% 이상은 TTS가 궁금하지, 그것을 운용시키는 무슨 Pytorch라던지, GAN, KSS, LJSpeech 같은 건 들어보지도 않았을 뿐더러 알기도 싫으실 겁니다.

그래서 간단히 이 사이트 사용법에 대해서 설명을 하자면, TTS처럼 글자 입력해서 목소리가 나오는 건 절대 아닙니다. (gTTS를 쓰면 되긴 하지만, 귀찮아서 하진 않습니다.)

따라서 여러분의 아무 말이나 하는 WAV 형식의 목소리 파일 (50건 이상 권장)과 말하고자 하는 문장이 포함된 WAV 형식의 목소리 파일 1건을 가져오셔서 진행하시면 되겠습니다.

아래의 방법을 따라주세요!

**✨ 추천 링크:**
* 유튜브 영상에서 목소리를 추출해서 TTS를 제작하는 Colab [-> (Chris140님 감사합니다)](https://colab.research.google.com/github/papercore-dev/perfectly-jogyo/blob/main/transcribe.ipynb)
* 여러가지 보이스들 [->](https://github.com/papercore-dev/perfectly-jogyo/tree/main/models)

##🚀 **TTS 모델 설정하기**
*숨겨진 셀 10개* 라고 적힌 부분 왼쪽의 큼지막한 재생 버튼을 눌러서 실행하시면 됩니다.

보안 관련 경고가 나타나면은 **무시하고 계속**을 누르시면 됩니다.

### 🐸TTS 설정하기
Coqui라는 회사에서 제작한 TTS인데요, Coqui는 음성 인식 및 음성 합성을 주 업무로 하고 있는 회사에요.


In [ ]:
!git clone https://github.com/Edresson/Coqui-TTS -b multilingual-torchaudio-SE TTS
!pip install -q -e TTS/
!pip install -q torchaudio==0.9.0

###✨ TTS 사전 설정값 다운로드
사전 설정값, 즉 체크포인트 없이는 GPU 사용량이 높은 트레이닝 과정을 거쳐야 하는데, 이거 하시는 분들은 그런거 잘 모르실 거잖아요? 그래서 미리 해봤습니다 :)

In [ ]:
# 사전 설정값 (체크포인트)을 구글 드라이브에서 받습니다.

# download config  
!gdown --id 1-PfXD66l1ZpsZmJiC-vhL055CDSugLyP
# download language json 
! gdown --id 1_Vb2_XHqcC0OcvRF82F883MTxfTRmerg
# download speakers json
! gdown --id 1SZ9GE0CBM-xGstiXH2-O2QWdmSXsBKdC -O speakers.json
# download checkpoint
! gdown --id 1sgEjHt0lbPSEw9-FSbC_mBoOPwNi87YR -O best_model.pth.tar

### ✨ 필요한 라이브러리 가져오기

In [ ]:
import sys
TTS_PATH = "TTS/"

# add libraries into environment
sys.path.append(TTS_PATH) # set this if TTS is not installed globally

import os
import string
import time
import argparse
import json

import numpy as np
import IPython
from IPython.display import Audio


import torch

from TTS.tts.utils.synthesis import synthesis
from TTS.tts.utils.text.symbols import make_symbols, phonemes, symbols
try:
  from TTS.utils.audio import AudioProcessor
except:
  from TTS.utils.audio import AudioProcessor


from TTS.tts.models import setup_model
from TTS.config import load_config
from TTS.tts.models.vits import *

### 경로 설정

In [ ]:
OUT_PATH = 'out/'

# create output path
os.makedirs(OUT_PATH, exist_ok=True)

# model vars 
MODEL_PATH = 'best_model.pth.tar'
CONFIG_PATH = 'config.json'
TTS_LANGUAGES = "language_ids.json"
TTS_SPEAKERS = "speakers.json"
USE_CUDA = torch.cuda.is_available()

### 모델 복원

In [ ]:
# load the config
C = load_config(CONFIG_PATH)


# load the audio processor
ap = AudioProcessor(**C.audio)

speaker_embedding = None

C.model_args['d_vector_file'] = TTS_SPEAKERS
C.model_args['use_speaker_encoder_as_loss'] = False

model = setup_model(C)
model.language_manager.set_language_ids_from_file(TTS_LANGUAGES)
# print(model.language_manager.num_languages, model.embedded_language_dim)
# print(model.emb_l)
cp = torch.load(MODEL_PATH, map_location=torch.device('cpu'))
# remove speaker encoder
model_weights = cp['model'].copy()
for key in list(model_weights.keys()):
  if "speaker_encoder" in key:
    del model_weights[key]

model.load_state_dict(model_weights)


model.eval()

if USE_CUDA:
    model = model.cuda()

# synthesize voice
use_griffin_lim = False

##🚀 **음성 인코더 설정**
음성 합성은 다 끝났는데, 결과물이 없다면 반쪽짜리 TTS가 되는지라, 인코더도 설정하려 해요.

아까처럼 ▶️ 버튼을 눌러주세요!

### ✨ 라이브러리 설치
음성의 질을 균등하게 해주는 FFMPEG Normalize와, 여러 음성을 합치는 Pydub을 설치해요.

In [ ]:
! pip install -q pydub ffmpeg-normalize

### ✨ 경로 설정
체크포인트 몇개만 더 받을게요!

In [ ]:
CONFIG_SE_PATH = "config_se.json"
CHECKPOINT_SE_PATH = "SE_checkpoint.pth.tar"

# download config 
! gdown --id  19cDrhZZ0PfKf2Zhr_ebB-QASRw844Tn1 -O $CONFIG_SE_PATH
# download checkpoint  
! gdown --id   17JsW6h6TIh7-LkU2EvB_gnNrPcdBxt7X -O $CHECKPOINT_SE_PATH

###Imports

In [ ]:
from TTS.tts.utils.speakers import SpeakerManager
from pydub import AudioSegment
from google.colab import files
import librosa

###Load the Speaker encoder

In [ ]:
SE_speaker_manager = SpeakerManager(encoder_model_path=CHECKPOINT_SE_PATH, encoder_config_path=CONFIG_SE_PATH, use_cuda=USE_CUDA)

###Define helper function

In [ ]:
def compute_spec(ref_file):
  y, sr = librosa.load(ref_file, sr=ap.sample_rate)
  spec = ap.spectrogram(y)
  spec = torch.FloatTensor(spec).unsqueeze(0)
  return spec

##🚀 **음성 합성하기**
자~ 이제 설정이 끝났습니다!
이제부터 음성 합성을 하고자 하는데요,

이름과 달리 그리 어렵지 않습니다.
한번 직접 해봐요!

###✨음성 업로드

In [ ]:
#@title 🎤 **새 보이스 제작하기**<br>원하는 보이스가 없다면, 새로운 보이스를 제작해도 좋아요.<br>**보이스 파일의 형식은 WAV (파형 오디오) 형식의 파일 필수, 50건 이상의 파일 권장 (좋은 결과를 얻을 수 있어요)입니다.**

print("변조하고자 하는 이의 원본 목소리를 업로드합니다.:")
target_files = files.upload()
target_files = list(target_files.keys())
for sample in target_files:
    !ffmpeg-normalize $sample -nt rms -t=-27 -o $sample -ar 16000 -f

In [ ]:
#@title 🎤 **아무 말이나 올리기**<br>자신의 보이스를 올리세요.<br>**보이스 파일의 형식은 WAV (파형 오디오) 형식의 파일 필수, 50건 이상의 파일 권장 (좋은 결과를 얻을 수 있어요)입니다.**
print("Select driving speaker reference audios files:")
driving_files = files.upload()
driving_files = list(driving_files.keys())
for sample in driving_files:
    !ffmpeg-normalize $sample -nt rms -t=-27 -o $sample -ar 16000 -f

In [ ]:
#@title 🎤 **대상이 할 말을 올리기**<br>**보이스 파일의 형식은 WAV (파형 오디오) 형식의 파일 필수, 1건의 파일 필수입니다.**

print("변조하고 싶은 말을 올리세요.")
driving_file = files.upload()
driving_file = list(driving_file.keys())
for sample in driving_file:
    !ffmpeg-normalize $sample -nt rms -t=-27 -o $sample -ar 16000 -f
driving_file = driving_file[0]

###✨ AI의 학습 단계

In [ ]:
#@title 🎤 **새 보이스 학습하기**<br>보이스 학습이 완료되었다면, 이 값을 복사해서, <a href="https://github.com/papercore-dev/perfectly-jogyo/issues/new">깃허브 이슈를 제작</a>하여 누구나 사용할 수 있게 도와주시면 감사하겠습니다.

target_emb = SE_speaker_manager.compute_d_vector_from_clip(target_files)
target_emb = torch.FloatTensor(target_emb).unsqueeze(0)
print('보이스 값입니다. 다음 줄부터 복사해주세요! \n')
print(target_emb)

In [ ]:
#@title 🎤 **내 보이스 학습하고 합성하기**

target_emb = input ("보이스 txt파일의 값을 입력하세요 :")

if driving_emb is None:
    driving_emb = SE_speaker_manager.compute_d_vector_from_clip(driving_files)
    driving_emb = torch.FloatTensor(driving_emb).unsqueeze(0)

driving_spec = compute_spec(driving_file)
y_lengths = torch.tensor([driving_spec.size(-1)])
if USE_CUDA:
    ref_wav_voc, _, _ = model.voice_conversion(driving_spec.cuda(), y_lengths.cuda(), driving_emb.cuda(), target_emb.cuda())
    ref_wav_voc = ref_wav_voc.squeeze().cpu().detach().numpy()
else:
    ref_wav_voc, _, _ = model.voice_conversion(driving_spec, y_lengths, driving_emb, target_emb)
    ref_wav_voc = ref_wav_voc.squeeze().detach().numpy()


print("결과물입니다:")
IPython.display.display(Audio(ref_wav_voc, rate=ap.sample_rate))